# main functions

In [4]:
import random
import numpy as np
from collections import defaultdict
from typing import List, Tuple

# ---------------------------------------------------------------------------
# Configurazioni e costanti

# Definizione degli operatori
UNARY_OPERATORS = ['sin', 'cos', 'exp', 'log', 'sqrt', 'tan', 'tanh', 'sinh', 'cosh', 'abs', 'log10', 'log2']
BINARY_OPERATORS = ['+', '-', '*', '/', '**', 'mod']

OPERATOR_WEIGHTS = {
    '+': 0.3,
    '-': 0.3,
    '*': 0.2,
    '/': 0.1,    # Divisioni meno frequenti
    '**': 0.05,  # Potenze rare
    'sin': 0.15,
    'cos': 0.15,
    'exp': 0.05, # Exponential molto raro
    'log': 0.1,
    'sqrt': 0.1,
    'tan': 0.05,
    'tanh': 0.05,
    'sinh': 0.05,
    'cosh': 0.05,
    'abs': 0.05,
    'log10': 0.05,
    'log2': 0.05,
    'mod': 0.05
}

class MutationConfig:
    MUTATION_WEIGHTS = {
        'SUBTREE': 0.4,
        'OPERATOR': 0.4,
        'VALUE': 0.2,
    }
    SUBTREE_DEPTH_RANGE = (1, 3)
    VALUE_STEP_FACTOR = 0.1
    MUTATION_DECAY = 0.5

# ---------------------------------------------------------------------------
# Definizione della classe Node
class Node:
    def __init__(self, value=None, op=None, left: 'Node' = None, right: 'Node' = None):
        self.value = value    # Per foglie: numero costante o variabile (es. "x[0]")
        self.op = op          # Per nodi interni: operatore (es. "+", "sin", "mod", ecc.)
        self.left = left
        self.right = right

    def evaluate(self, x: np.ndarray):
        """
        Valuta l'albero (funzione simbolica) sul vettore x.
        (Implementa l'uso di SAFE_OPERATIONS e clip_value secondo il tuo contesto.)
        """
        try:
            # Valutazione per operatori binari
            if self.op in BINARY_OPERATORS:
                if not self.left or not self.right:
                    return 0
                left_val = self.left.evaluate(x)
                right_val = self.right.evaluate(x)
                return SAFE_OPERATIONS[self.op](left_val, right_val)
            # Valutazione per operatori unari
            elif self.op in UNARY_OPERATORS:
                if not self.left:
                    return 0
                operand_val = self.left.evaluate(x)
                return SAFE_OPERATIONS[self.op](operand_val)
            # Se si tratta di una foglia con variabile
            elif isinstance(self.value, str) and self.value.startswith("x["):
                index = int(self.value[2:-1])
                return clip_value(x[index])
            elif self.value is not None:
                return clip_value(self.value)
        except Exception:
            return 0

    def copy(self) -> 'Node':
        """Restituisce una copia profonda dell'albero."""
        return Node(
            value=self.value,
            op=self.op,
            left=self.left.copy() if self.left else None,
            right=self.right.copy() if self.right else None,
        )

    def __str__(self) -> str:
        """Restituisce una rappresentazione in stringa dell'albero."""
        if self.op:
            if self.left and self.right:
                return f"({self.left} {self.op} {self.right})"
            elif self.left:
                return f"{self.op}({self.left})"
        return str(self.value)

# ---------------------------------------------------------------------------
# Funzioni "safe" e utilità matematiche
CLIP_MIN = -1e6
CLIP_MAX = 1e6

def clip_value(value):
    return np.clip(value, CLIP_MIN, CLIP_MAX)

def safe_divide(a, b):
    return a / b if b != 0 else 0

def safe_sin(x):
    x = np.clip(x, -1000, 1000)
    result = np.sin(x)
    return np.clip(result, -1000, 1000)

def safe_cos(x):
    x = np.clip(x, -1000, 1000)
    result = np.cos(x)
    return np.clip(result, -1000, 1000)

def safe_sinh(x):
    x = np.clip(x, -100, 100)
    result = np.sinh(x)
    return np.clip(result, -1000, 1000)

def safe_cosh(x):
    x = np.clip(x, -100, 100)
    result = np.cosh(x)
    return np.clip(result, -1000, 1000)

def safe_tan(x):
    if np.isclose(np.mod(x, np.pi), np.pi / 2):
        return float(10**6)
    x = np.clip(x, -1000, 1000)
    result = np.tan(x)
    return np.clip(result, -1000, 1000)

def safe_log10(x):
    if x <= 0:
        return float(10**6)
    result = np.log10(x)
    return np.clip(result, -1000, 1000)

def safe_pow(base, exp):
    base = np.clip(base, -1000, 1000)
    exp = np.clip(exp, -100, 100)
    if base == 0 and exp < 0:
        return float(10**6)
    elif base < 0 and not np.all(np.isinteger(exp)):
        return float(10**6)
    try:
        result = np.power(base, exp)
        return np.clip(result, -1000, 1000)
    except ValueError:
        return float(10**6)

def safe_log2(x):
    if x <= 0:
        return float(10**6)
    result = np.log2(x)
    return np.clip(result, -1000, 1000)

def safe_mod(x, y):
    if y == 0:
        return float(10**6)
    x = np.clip(x, -1000, 1000)
    y = np.clip(y, -1000, 1000)
    result = np.mod(x, y)
    return np.clip(result, -1000, 1000)

def safe_tanh(x):
    x = np.clip(x, -1000, 1000)
    result = np.tanh(x)
    return np.clip(result, -1000, 1000)

def safe_exp(x):
    x = np.clip(x, -100, 100)
    result = np.exp(x)
    return np.clip(result, -1000, 1000)

def safe_log(x):
    if x <= 0:
        return float(10**6)
    result = np.log(x)
    return np.clip(result, -1000, 1000)

def safe_sqrt(x):
    x = np.maximum(x, 0)
    result = np.sqrt(x)
    return np.clip(result, -1000, 1000)

def safe_abs(x):
    result = np.abs(x)
    return np.clip(result, -1000, 1000)

SAFE_OPERATIONS = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b,
    '/': safe_divide,
    'sin': safe_sin,
    'cos': safe_cos,
    'exp': safe_exp,
    'log': safe_log,
    'sqrt': safe_sqrt,
    'tan': safe_tan,
    'tanh': safe_tanh,
    'sinh': safe_sinh,
    'cosh': safe_cosh,
    'abs': safe_abs,
    'log10': safe_log10,
    'log2': safe_log2,
    '**': safe_pow,
    'mod': safe_mod
}

# ---------------------------------------------------------------------------
# Helper functions per la manipolazione degli alberi

def get_random_node(node: Node, include_root: bool = True) -> Node:
    """
    Raccoglie ricorsivamente tutti i nodi dell'albero e ne restituisce uno casuale.
    Se include_root=False, esclude la radice (se possibile).
    """
    nodes = []
    def traverse(n: Node):
        if n is None:
            return
        nodes.append(n)
        traverse(n.left)
        traverse(n.right)
    traverse(node)
    if not include_root and len(nodes) > 1:
        nodes = nodes[1:]
    return random.choice(nodes)

def get_random_node_with_parent(root: Node) -> Tuple[Node, Node, bool]:
    """
    Restituisce una tupla (nodo, genitore, is_left) scelta casualmente,
    dove is_left è True se il nodo è figlio sinistro del genitore.
    """
    nodes = []
    def traverse(node: Node, parent=None, is_left: bool = None):
        if node is None:
            return
        nodes.append((node, parent, is_left))
        traverse(node.left, node, True)
        traverse(node.right, node, False)
    traverse(root)
    return random.choice(nodes)

def replace_node(root: Node, target: Node, new_subtree: Node) -> bool:
    """
    Sostituisce ricorsivamente il nodo target con new_subtree nell'albero radicato in root.
    Restituisce True se la sostituzione ha avuto successo.
    """
    if root is None:
        return False
    if root.left == target:
        root.left = new_subtree
        return True
    if root.right == target:
        root.right = new_subtree
        return True
    return replace_node(root.left, target, new_subtree) or replace_node(root.right, target, new_subtree)

def get_random_leaf_with_parent(root: Node) -> Tuple[Node, Node, bool]:
    """
    Restituisce una tupla (foglia, genitore, is_left) scegliendo casualmente una foglia (nodo senza figli).
    """
    leaves = []
    def traverse(node: Node, parent=None, is_left: bool = None):
        if node is None:
            return
        if node.left is None and node.right is None:
            leaves.append((node, parent, is_left))
        else:
            traverse(node.left, node, True)
            traverse(node.right, node, False)
    traverse(root)
    if leaves:
        return random.choice(leaves)
    return (None, None, None)

# ---------------------------------------------------------------------------
# Operatori Genetici: Crossover e Mutazioni

def crossover(parent1: Node, parent2: Node) -> Node:
    """
    Crossover per symbolic regression: scambia sottoalberi tra i genitori.
      - Crea una copia di parent1 (offspring).
      - Seleziona un nodo casuale (target) in offspring.
      - Seleziona un nodo casuale in parent2 (donor).
      - Sostituisce il sottoalbero target in offspring con una copia del donor.
    """
    offspring = parent1.copy()
    target = get_random_node(offspring, include_root=True)
    donor = get_random_node(parent2, include_root=True)
    new_subtree = donor.copy()
    if offspring == target:
        offspring = new_subtree
    else:
        replace_node(offspring, target, new_subtree)
    return offspring

def subtree_mutation(individual: Node, max_depth: int, n_variables: int) -> Node:
    """
    Subtree mutation:
      - Seleziona un nodo casuale nell'individuo.
      - Genera un nuovo sottoalbero casuale (con profondità scelta randomicamente).
      - Sostituisce il nodo selezionato con il nuovo sottoalbero.
    """
    mutated = individual.copy()
    target = get_random_node(mutated, include_root=True)
    new_depth = random.randint(MutationConfig.SUBTREE_DEPTH_RANGE[0],
                               MutationConfig.SUBTREE_DEPTH_RANGE[1])
    new_subtree = create_random_tree(0, new_depth, n_variables)
    if mutated == target:
        mutated = new_subtree
    else:
        replace_node(mutated, target, new_subtree)
    return mutated

def point_mutation(individual: Node, n_variables: int) -> Node:
    """
    Point mutation:
      - Seleziona un singolo nodo e lo modifica.
      - Se il nodo contiene un operatore, lo sostituisce con un altro compatibile,
        mantenendo la compatibilità (es. operatore binario mantiene due figli).
      - Se il nodo è una foglia, modifica leggermente il valore (o l'indice se è una variabile).
    """
    mutated = individual.copy()
    target = get_random_node(mutated, include_root=True)
    if target.op is not None:
        # Se il nodo ha un operatore, valuta se è binario o unario
        if target.left is not None and target.right is not None:
            possible_ops = [op for op in OPERATOR_WEIGHTS.keys() if op not in UNARY_OPERATORS]
        elif target.left is not None and target.right is None:
            possible_ops = [op for op in OPERATOR_WEIGHTS.keys() if op in UNARY_OPERATORS]
        else:
            possible_ops = list(OPERATOR_WEIGHTS.keys())
        if target.op in possible_ops and len(possible_ops) > 1:
            possible_ops.remove(target.op)
        target.op = random.choice(possible_ops)
    else:
        # Se il nodo è una foglia: se è numerico, modifica il valore; se è una variabile, cambia indice.
        if isinstance(target.value, (int, float)):
            factor = MutationConfig.VALUE_STEP_FACTOR
            target.value += random.uniform(-factor, factor) * (target.value if target.value != 0 else 1)
        elif isinstance(target.value, str) and target.value.startswith("x["):
            new_index = random.randint(0, n_variables - 1)
            target.value = f"x[{new_index}]"
    return mutated

def hoist_mutation(individual: Node) -> Node:
    """
    Hoist mutation:
      - Seleziona un sottoalbero (escludendo la radice) e lo promuove a nuovo individuo.
      - Questo operatore tende a semplificare l'albero.
    """
    mutated = individual.copy()
    target = get_random_node(mutated, include_root=False)
    return target.copy()

def expansion_mutation(individual: Node, max_depth: int, n_variables: int) -> Node:
    """
    Expansion mutation:
      - Seleziona una foglia e la sostituisce con un nuovo sottoalbero casuale.
      - Aumenta la complessità dell'albero, trasformando una foglia in un nodo interno.
    """
    mutated = individual.copy()
    target, parent, is_left = get_random_leaf_with_parent(mutated)
    if target is None:
        return mutated
    new_depth = random.randint(MutationConfig.SUBTREE_DEPTH_RANGE[0],
                               MutationConfig.SUBTREE_DEPTH_RANGE[1])
    new_subtree = create_random_tree(0, new_depth, n_variables)
    if parent is None:
        mutated = new_subtree
    else:
        if is_left:
            parent.left = new_subtree
        else:
            parent.right = new_subtree
    return mutated

def collapse_mutation(individual: Node) -> Node:
    """
    Collapse mutation:
      - Seleziona un nodo interno (sottoalbero) e lo sostituisce con una foglia.
      - Semplifica l'albero, contrastando il fenomeno del bloat.
    """
    mutated = individual.copy()
    nodes = []
    def traverse(node: Node, parent=None, is_left: bool = None):
        if node is None:
            return
        if node.left is not None or node.right is not None:
            nodes.append((node, parent, is_left))
        traverse(node.left, node, True)
        traverse(node.right, node, False)
    traverse(mutated)
    if not nodes:
        return mutated
    target, parent, is_left = random.choice(nodes)
    new_leaf = Node(value=random.uniform(-10, 10))
    if parent is None:
        mutated = new_leaf
    else:
        if is_left:
            parent.left = new_leaf
        else:
            parent.right = new_leaf
    return mutated

# ---------------------------------------------------------------------------
# Funzioni per la generazione casuale dell'albero

def generate_constant():
    """
    Genera un valore costante casuale.
      - Con probabilità 0.5 restituisce un valore piccolo (tra -1 e 1),
        altrimenti un valore più ampio (tra -10 e 10).
    """
    if random.random() < 0.5:
        return random.uniform(-1, 1)
    else:
        return random.uniform(-10, 10)

def create_random_tree(depth: int, max_depth: int, n_variables: int) -> Node:
    """
    Crea un albero casuale per la symbolic regression.
      - Se la profondità raggiunge max_depth o in base a una probabilità,
        genera un nodo foglia (variabile o costante).
      - Altrimenti, genera un nodo operatore (binario o unario) e ricorsivamente crea i figli.
    """
    if depth >= max_depth or (depth > 0 and random.random() < 0.5):
        # Nodo foglia: probabilità di scegliere una variabile oppure una costante
        if random.random() < 0.7:
            return Node(value=f"x[{random.randint(0, n_variables - 1)}]")
        else:
            return Node(value=generate_constant())
    # Genera un nodo operatore
    op = random.choices(list(OPERATOR_WEIGHTS.keys()),
                        weights=list(OPERATOR_WEIGHTS.values()),
                        k=1)[0]
    if op in BINARY_OPERATORS:
        left = create_random_tree(depth + 1, max_depth, n_variables)
        right = create_random_tree(depth + 1, max_depth, n_variables)
        return Node(op=op, left=left, right=right)
    elif op in UNARY_OPERATORS:
        operand = create_random_tree(depth + 1, max_depth, n_variables)
        return Node(op=op, left=operand)
    else:
        # In casi rari, se l'operatore non è categorizzato, lo tratta come unario.
        operand = create_random_tree(depth + 1, max_depth, n_variables)
        return Node(op=op, left=operand)


def tournament_selection(population: List[Node], fitness_scores: List[float], tournament_size: int = 3) -> Node:
    """Selezione tramite torneo."""
    competitors = random.sample(list(zip(population, fitness_scores)), tournament_size)
    winner = min(competitors, key=lambda x: x[1])  # Migliore fitness
    return winner[0]

def calculate_fitness(tree: Node, x: np.ndarray, y_true: np.ndarray) -> float:
    """Calcola la fitness di un individuo basata sull'errore quadratico medio."""
    try:
        y_pred = np.array([tree.evaluate(x[:, i]) for i in range(x.shape[1])])
        y_pred = np.clip(y_pred, CLIP_MIN, CLIP_MAX)
        y_pred = np.nan_to_num(y_pred, nan=0.0, posinf=CLIP_MAX, neginf=CLIP_MIN)  # Gestione di nan e inf
        return np.mean(np.square(y_true-y_pred))
    except Exception:
        return float('inf')


def tree_to_string(node):
    """
    Converte ricorsivamente un albero (istanza di Node) in una stringa 
    che rappresenta un'espressione Python valida.  
    In particolare:
      - Le funzioni matematiche vengono prefissate da 'np.'  
      - L'operatore 'mod' viene tradotto in 'np.mod(a, b)'  
      - Se un nodo è mancante, non viene inserito 'None'.
    """
    # Mappa degli operatori alla loro rappresentazione in Python
    op_map = {
        '+': '+',
        '-': '-',
        '*': '*',
        '/': '/',
        '**': '**',
        'sin': 'np.sin',
        'cos': 'np.cos',
        'exp': 'np.exp',
        'log': 'np.log',
        'sqrt': 'np.sqrt',
        'tan': 'np.tan',
        'tanh': 'np.tanh',
        'sinh': 'np.sinh',
        'cosh': 'np.cosh',
        'abs': 'np.abs',
        'log10': 'np.log10',
        'log2': 'np.log2',
        'mod': 'np.mod'
    }
    
    if node is None:
        return ""
    
    # Se il nodo è una foglia (nessun operatore) e il suo valore non è None
    if node.op is None:
        return str(node.value) if node.value is not None else ""
    
    # Per i nodi operativi, definiamo in modo sicuro le stringhe per i figli
    left_str = tree_to_string(node.left) if node.left is not None else ""
    right_str = tree_to_string(node.right) if node.right is not None else ""
    
    # Se entrambi i figli sono presenti, assumiamo un operatore binario
    if node.left is not None and node.right is not None:
        # Per l'operatore 'mod' usiamo la sintassi a funzione, ad esempio: np.mod(a, b)
        if node.op == 'mod':
            return f"{op_map[node.op]}({left_str}, {right_str})"
        # Per gli operatori standard li inseriamo tra parentesi
        return f"({left_str} {op_map[node.op]} {right_str})"
    
    # Se solo il figlio sinistro esiste (operatori unari)
    if node.left is not None:
        return f"{op_map[node.op]}({left_str})"
    
    # Se per qualche motivo non abbiamo figli, restituisco il valore (sempre evitando None)
    return str(node.value) if node.value is not None else ""

# ---------------------------------------------------------------------------
# Fine del modulo


In [10]:

# --- Funzione helper per applicare una mutazione scelta tra quelle implementate ---
def apply_mutation(individual, mutation_prob, max_depth, n_variables):
    """
    Applica una mutazione all'individuo con probabilità mutation_prob.
    Viene scelta casualmente (o in base a pesi, se desiderato) una tra le funzioni di mutazione:
      - subtree_mutation (richiede max_depth e n_variables)
      - point_mutation (richiede n_variables)
      - hoist_mutation (non richiede parametri extra)
      - expansion_mutation (richiede max_depth e n_variables)
      - collapse_mutation (non richiede parametri extra)
    """
    if random.random() > mutation_prob:
        return individual  # Nessuna mutazione

    mutation_functions = [
        subtree_mutation,
        point_mutation,
        hoist_mutation,
        # expansion_mutation,
        collapse_mutation
    ]
    # Scegliamo una funzione di mutazione a caso
    chosen_mutation = random.choice(mutation_functions)
    
    # Chiamata in base alla funzione scelta:
    if chosen_mutation in [subtree_mutation, expansion_mutation]:
        return chosen_mutation(individual, max_depth, n_variables)
    elif chosen_mutation == point_mutation:
        return chosen_mutation(individual, n_variables)
    else:
        # hoist_mutation e collapse_mutation non richiedono max_depth o n_variables
        return chosen_mutation(individual)


# main algorithm

In [19]:
import os
import glob


def genetic_algorithm(filepath: str):
    """
    Esegue l'algoritmo genetico su tutti i file 'problem_#.npz' presenti nella cartella
    indicata da `filepath` e salva le funzioni ottenute in un file Python denominato 's323914.py'.
    
    La funzione scrive nel file una definizione per ciascun problema, nel formato:
    
        import numpy as np
        
        def f1(x: np.ndarray) -> np.ndarray:  # mse: 0.1234
            return <formula>
    
    Dove il numero nella funzione (f1, f2, …) corrisponde al numero estratto dal nome del file
    (es. "problem_1.npz") e il commento indica il valore della fitness (mse) ottenuta.
    """
    # Costanti dell'algoritmo
    POPULATION_SIZE = 500
    MAX_DEPTH = 6
    GENERATIONS = 50
    # MIN_DIVERSITY = 0.6
    TOURNAMENT_SIZE = 3
    replacement_rate = 0.8
    num_replacement = int(POPULATION_SIZE * replacement_rate)
    
    # Trova tutti i file con pattern "problem_*.npz" nella cartella specificata
    problem_files = glob.glob(os.path.join(filepath, "problem_*.npz"))
    solutions = []  # Lista dei tuple: (numero_problema, best_fitness, best_code)

    # Ciclo sui file dei problemi (ordinati per numero)
    for prob_file in sorted(problem_files):
        
        base = os.path.basename(prob_file)
        try:
            num_str = base.split('_')[1].split('.')[0]
            prob_number = int(num_str)
        except (IndexError, ValueError):
            print(f"Nome file non conforme, salto: {prob_file}")
            continue

        print(f"Risoluzione del problema {prob_number}...")
        # Escludi il problem_0
        if prob_number == 1 or prob_number == 2 or prob_number ==  5:
            print(f"Salto il problema {prob_number} (problem_0 escluso).")
            continue

        # Carica i dati dal file .npz
        data = np.load(prob_file)
        x_train = data['x']
        y_train = data['y']
        N_VARIABLES = x_train.shape[0]

        # Inizializza la popolazione
        population = [create_random_tree(0, MAX_DEPTH, N_VARIABLES) for _ in range(POPULATION_SIZE)]

        best_fitness = float('inf')
        best_tree = None

        fitness_scores = [calculate_fitness(ind, x_train, y_train) for ind in population]

        # Ciclo evolutivo
        for generation in range(GENERATIONS):
            next_generation = []
            for _ in range(0, POPULATION_SIZE, 2):
                p1 = tournament_selection(population, fitness_scores, TOURNAMENT_SIZE)
                p2 = tournament_selection(population, fitness_scores, TOURNAMENT_SIZE)
                child1 = crossover(p1, p2)
                child2 = crossover(p2, p1)
                child1 = apply_mutation(child1, mutation_prob=0.3, max_depth=MAX_DEPTH, n_variables=N_VARIABLES)
                child2 = apply_mutation(child2, mutation_prob=0.3, max_depth=MAX_DEPTH, n_variables=N_VARIABLES)
                next_generation.extend([child1, child2])
            offspring_fitness = [calculate_fitness(ind, x_train, y_train) for ind in next_generation]

            # Sostituisce alcuni individui della popolazione con i figli se migliorano la fitness
            for child, fitness in zip(next_generation[:num_replacement], offspring_fitness[:num_replacement]):
                least_fit_idx = np.argmax(fitness_scores)
                if fitness < fitness_scores[least_fit_idx]:
                    population[least_fit_idx] = child
                    fitness_scores[least_fit_idx] = fitness
                    if fitness < best_fitness:
                        best_fitness = fitness
                        best_tree = child

            diversity = calculate_population_diversity(population)
            print(f"Diversità: {diversity:.2f}")
            # # Se ogni 5 generazioni la diversità scende sotto la soglia, si applica una strategia per mantenerla
            # if generation % 2 == 0:
            #     if diversity < MIN_DIVERSITY:
            #         population = maintain_diversity(population, MIN_DIVERSITY, MAX_DEPTH, N_VARIABLES)
            print(f"Generazione {generation + 1}: Miglior fitness: {best_fitness}, Miglior individuo: {best_tree}")

        print(f"Problema {prob_number} - Miglior individuo: {best_tree} con fitness: {best_fitness}")
        # Converte l'albero dell'individuo migliore in una stringa contenente una formula valida in Python.
        best_code = tree_to_string(best_tree)
        solutions.append((prob_number, best_fitness, best_code))

    # Scrive il file di output "s323914.py" con le funzioni ottenute
    output_filename = "s323914.py"
    with open(output_filename, "w") as f:
        f.write("import numpy as np\n\n")
        # Per ogni problema scrive la definizione della funzione (f1, f2, …)
        for prob_number, best_fitness, best_code in sorted(solutions, key=lambda x: x[0]):
            f.write(f"def f{prob_number}(x: np.ndarray) -> np.ndarray:  # mse: {best_fitness:.4f}\n")
            f.write("    return " + best_code + "\n\n")
    print(f"Soluzioni salvate in {output_filename}")


In [ ]:


genetic_algorithm("data")


Risoluzione del problema 1...
Salto il problema 1 (problem_0 escluso).
Risoluzione del problema 2...
Salto il problema 2 (problem_0 escluso).
Risoluzione del problema 3...


C:\Users\silvi\AppData\Local\Temp\ipykernel_17900\318836476.py:151: RuntimeWarning: overflow encountered in power
  result = np.power(base, exp)


Generazione 1: Miglior fitness: 1731.7605610644266, Miglior individuo: ((sin(x[1]) - x[1]) * 5.38253128017379)
Generazione 2: Miglior fitness: 1549.3961121649388, Miglior individuo: cosh((tanh(log(x[1])) - x[1]))
Generazione 3: Miglior fitness: 755.6150654751211, Miglior individuo: ((sqrt(0.7725136750847563) - x[1]) * log(log(-0.5841877718344692)))
Generazione 4: Miglior fitness: 755.6150654751211, Miglior individuo: ((sqrt(0.7725136750847563) - x[1]) * log(log(-0.5841877718344692)))
Generazione 5: Miglior fitness: 706.2525115891408, Miglior individuo: ((sqrt(3.1372718810452582) - x[1]) * log(log(-0.5841877718344692)))
Generazione 6: Miglior fitness: 706.2525115891408, Miglior individuo: ((sqrt(3.1372718810452582) - x[1]) * log(log(-0.5841877718344692)))
Generazione 7: Miglior fitness: 706.2525115891408, Miglior individuo: ((sqrt(3.1372718810452582) - x[1]) * log(log(-0.5841877718344692)))
Generazione 8: Miglior fitness: 706.2525115891408, Miglior individuo: ((sqrt(3.1372718810452582) 